In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from nltk.corpus import stopwords
import re
# from wordcloud import WordCloud

ModuleNotFoundError: No module named 'wordcloud'

In [3]:
# conda install -c conda-forge wordcloud=1.2.1

SyntaxError: invalid syntax (<ipython-input-3-cde86110a550>, line 1)

In [ ]:
data = pd.read_csv("Tweets_source_data.csv")

In [ ]:
data

# first model

In [ ]:
# https://www.kaggle.com/maniche04/sentiment-analysis-scikit

In [ ]:
airwords = ['airline','air','plane','flight']
airlines = data['airline']

stop = stopwords.words('english')


In [ ]:
def preprocessor(text):
    words = text.split()
    del words[0]
    words = [re.sub('[^0-9a-zA-Z]+', '', w.replace('#','').lower()) for w in words if not w.startswith('http') and not w.startswith('@')]
    finalwords = [w for w in words if (not w.isdigit() and w not in airwords and w not in airlines)]
    return ' '.join(finalwords)

In [ ]:
#print(data.head(3))
data = data.loc[:14000,['text','airline_sentiment']]
data['text'] = data['text'].apply(preprocessor)

print(preprocessor("This is a tweet flight 979 and i'm happy #flight #travel @maniche04"))


In [ ]:
test_data = data[4200:]
train_data = data[:9800]

print(train_data['text'][91:120])
#print(test_data['text'])

In [ ]:
# extracting the feature list
from sklearn.feature_extraction.text import CountVectorizer
#count = CountVectorizer(ngram_range=(1,2))
#bag = count.fit_transform(s for s in data.loc[:,'text'])
#print(count.vocabulary_)

# tranforming the feature list
from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer = tf_transformer.fit_transform(bag)

# training
from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB().fit(bag,data['airline_sentiment'])

# implement pipeline
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer(ngram_range=(4,5))),('tfidf',TfidfTransformer()),('clf',MultinomialNB())])
text_clf = text_clf.fit(train_data['text'],train_data['airline_sentiment'])

print('Test Accuracy: %.3f' % text_clf.score(test_data['text'], test_data['airline_sentiment']))

# 2nd model

In [ ]:
# Check the ratio of positive and negative tweets for each airline
data = pd.read_csv("Tweets_source_data.csv")

data['countval']=1
groupby_object=data[['airline','airline_sentiment','countval']].groupby(['airline','airline_sentiment']).aggregate(sum)
groupby_object.unstack(level=1).plot(kind='bar')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#####
data['text']=data['text'].map(lambda x: re.sub("^@[^\s]+\s","",x))
def getHashtag(x):
	g=re.match("^[^#]+#([^\s]+).*",x)
	if g:
		return g.group(1)
	else:
		return ""
	
data['hashtags']=data['text'].map(getHashtag)
# Convert to lower case
data['hashtags']=data['hashtags'].str.lower() 

from nltk.corpus import stopwords
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = raw_review
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

Reviews=[]
dataPos=data[data['airline_sentiment']=='positive']
for i in range(0, len(dataPos)):
    Reviews.append(review_to_words(dataPos['text'].tolist()[i]))

vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word',stop_words='english')
vect.fit(Reviews)
idf = vect._tfidf.idf_
wordDict=dict(zip(vect.get_feature_names(), idf))

In [ ]:
word1={k: v for k, v in wordDict.items() if v < 4}
from wordcloud import WordCloud
wordcloud = WordCloud().generate(' '.join(word1.keys()))
plt.title('POSITIVE : TFID FREQ < 4')
plt.imshow(wordcloud)